In [1]:
import json
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer
import numpy as np
from transformers import RobertaTokenizer, RobertaModel,  RobertaForSequenceClassification
import torch.optim as optim
import tqdm
import  torch.nn.functional as F


## 参数设置

In [2]:
device ="cuda:1"
batch_size = 64

## 数据读取部分

In [3]:
class Seq2SeqDataset(Dataset):
    """
    A Simple Seq2Seq Dataset Implementation
    """
    def __init__(self, fact_filename, romantic_filename,funny_filename, tokenizer, add_bos_token=True, add_eos_token=True):
        data = []
        with open(fact_filename,'r') as f:
            line = f.readline()
            while line:
                data.append({"source":"","target":line.replace('\n',''),"style":"fact"})
                line = f.readline()

        with open(romantic_filename,'r') as f:
            line = f.readline()
            while line:
                data.append({"source":"","target":line.replace('\n',''),"style":"romantic"})
                line = f.readline()        

        with open(funny_filename,'r') as f:
            line = f.readline()
            while line:
                data.append({"source":"","target":line.replace('\n',''),"style":"funny"})
                line = f.readline()    

        self.data = data
        self.tokenizer = tokenizer
        self.add_bos_token = add_bos_token
        self.add_eos_token = add_eos_token

    def __getitem__(self, index):
        item = self.data[index]
        target_token_ids = self.tokenizer.encode(item["target"], add_special_tokens=False)

        if self.add_bos_token:
            target_token_ids.insert(0, self.tokenizer.bos_token_id)

        if self.add_eos_token:
            target_token_ids.append(self.tokenizer.eos_token_id)


        item["target_token_ids"] = torch.LongTensor(target_token_ids)
        
        if item["style"]=='fact':
            item["source_token_ids"] = [1,0,0]
        elif item["style"]=='romantic':
            item["source_token_ids"] = [0, 1, 0]
        elif item["style"]=='funny':
            item["source_token_ids"] = [0, 0, 1]
        return item

    def __len__(self):
        return len(self.data)

    def collate_fn(self, batch):
        new_batch = {}
        new_batch["source_token_ids"] = torch.tensor([item["source_token_ids"] for item in batch])
        new_batch["target_token_ids"] = pad_sequence(
            [item["target_token_ids"] for item in batch], batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        #sample_batch_size = len(new_batch["target_token_ids"])
        #past = torch.randn(size=(12, 2, sample_batch_size, 12, 1, 61))  # .to(self.device)  # 61=64-3
        #temp = new_batch["source_token_ids"].unsqueeze(1).unsqueeze(2).unsqueeze(0).unsqueeze(0)
        #classification = torch.tile(temp, (12, 2, 1, 12, 1, 1))
        #new_batch["past"] = torch.cat((classification, past), dim=-1)
        new_batch["style"] = [item["style"] for item in batch]
        return new_batch



In [4]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [5]:
fact_filename = "./StyleCaption/fact-train.txt"
romantic_filename = "./StyleCaption/romantic-train.txt"
funny_filename = "./StyleCaption/funny-train.txt"
train_dataset = Seq2SeqDataset(fact_filename,romantic_filename,funny_filename , tokenizer)
train_dataloader = DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_dataset.collate_fn)

In [6]:
fact_filename = "./StyleCaption/fact-val.txt"
romantic_filename = "./StyleCaption/romantic-val.txt"
funny_filename = "./StyleCaption/funny-val.txt"
valid_dataset = Seq2SeqDataset(fact_filename,romantic_filename,funny_filename , tokenizer)
valid_dataloader = DataLoader(
            valid_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_dataset.collate_fn)

In [7]:
fact_filename = "./StyleCaption/fact-test.txt"
romantic_filename = "./StyleCaption/romantic-test.txt"
funny_filename = "./StyleCaption/funny-test.txt"
test_dataset = Seq2SeqDataset(fact_filename,romantic_filename,funny_filename , tokenizer)
test_dataloader = DataLoader(
            test_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_dataset.collate_fn)

In [8]:
# for batch in train_dataloader:
#     print(batch)

## 模型准备

In [9]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base',num_labels=3)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [10]:
model = model.to(device)

In [11]:
epoch_n = 20
# num_train_optimization_steps = int(18000 / 32)

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
# lr_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=200, t_total=num_train_optimization_steps)

## 训练

In [13]:
acc_max = 0
for epoch in range(epoch_n):
    epoch_num = 0 
    epoch_real= 0
    n=0
    loss = 0
    for batch in tqdm.tqdm(train_dataloader):
    #for batch in train_dataloader:
        sequence = batch['target_token_ids'].to(device)
        label_onehot = batch['source_token_ids'] # n*3,tensor
        label = torch.argmax(label_onehot,dim=-1).to(device) # n tensor
        
        sequence_logits = model(sequence).logits
        
        sequence_cross_entropy_loss = F.cross_entropy(sequence_logits, label)
        sequence_cross_entropy_loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        #打印
        loss+=sequence_cross_entropy_loss
        n+=1
        batch_real = (torch.argmax(sequence_logits,dim=-1) == label).sum()
        batch_num = len(label)
        epoch_real += batch_real
        epoch_num += len(label)
#         if n ==1:
#         从这一行才看出来
#             print(sequence)
#             print(F.softmax( sequence_logits,dim=-1))
#             print(label)
#             print(torch.argmax(sequence_logits,dim=-1))
#             print(torch.argmax(sequence_logits,dim=-1) == label)
#             print((torch.argmax(sequence_logits,dim=-1) == label).sum())
#         #
#         print("{}-epoch| {}-th batch,准确率:{},loss:{}".format(epoch+1,n, batch_real/batch_num,sequence_cross_entropy_loss))
    acc_rate = epoch_real/epoch_num
    print("第{}轮epoch训练时在训练集上,正确个数：{},总个数：{},准确率:{},学习率: {},loss: {}"
          .format(epoch+1,epoch_real,epoch_num,acc_rate,optimizer.param_groups[0]['lr'],loss/n))
    #测试和验证
    with torch.no_grad():
        
        epoch_num = 0 
        epoch_real= 0
        for batch in test_dataloader:
            sequence = batch['target_token_ids'].to(device)
            label_onehot = batch['source_token_ids'] # n*3,tensor
            label = torch.argmax(label_onehot,dim=-1).to(device) # n tensor
            sequence_logits = model(sequence).logits
            #打印
            batch_real = (torch.argmax(sequence_logits,dim=-1) == label).sum()
            batch_num = len(label)
            epoch_real += batch_real
            epoch_num += len(label)
        acc_rate = epoch_real/epoch_num
        print("第{}轮epoch后，在测试集上,正确个数：{},总个数：{},准确率:{}"
          .format(epoch+1,epoch_real,epoch_num,acc_rate))
        
        epoch_num = 0 
        epoch_real= 0
        for batch in valid_dataloader:
            sequence = batch['target_token_ids'].to(device)
            label_onehot = batch['source_token_ids'] # n*3,tensor
            label = torch.argmax(label_onehot,dim=-1).to(device) # n tensor
            sequence_logits = model(sequence).logits
            #打印
            batch_real = (torch.argmax(sequence_logits,dim=-1) == label).sum()
            batch_num = len(label)
            epoch_real += batch_real
            epoch_num += len(label)
        acc_rate = epoch_real/epoch_num
        print("第{}轮epoch后，在验证集上,正确个数：{},总个数：{},准确率:{}"
          .format(epoch+1,epoch_real,epoch_num,acc_rate))
        
        if acc_rate>acc_max:
            acc_max  = acc_rate
            print("保存模型更新",'classify'+str(acc_rate)+'.pth')
            torch.save(model,'classify.pth')
        
        
        
        

100%|██████████| 282/282 [00:24<00:00, 11.61it/s]


第1轮epoch训练时在训练集上,正确个数：10641,总个数：18000,准确率:0.5911666750907898,学习率: 1e-05,loss: 0.8383419513702393
第1轮epoch后，在测试集上,正确个数：1121,总个数：1500,准确率:0.7473333477973938
第1轮epoch后，在验证集上,正确个数：1122,总个数：1500,准确率:0.7479999661445618
保存模型更新 classifytensor(0.7480, device='cuda:1').pth


100%|██████████| 282/282 [00:24<00:00, 11.46it/s]


第2轮epoch训练时在训练集上,正确个数：14839,总个数：18000,准确率:0.8243889212608337,学习率: 1e-05,loss: 0.4548899233341217
第2轮epoch后，在测试集上,正确个数：1166,总个数：1500,准确率:0.7773333191871643
第2轮epoch后，在验证集上,正确个数：1164,总个数：1500,准确率:0.7759999632835388
保存模型更新 classifytensor(0.7760, device='cuda:1').pth


100%|██████████| 282/282 [00:25<00:00, 11.28it/s]


...

第18轮epoch训练时在训练集上,正确个数：17390,总个数：18000,准确率:0.9661111235618591,学习率: 1e-05,loss: 0.10167549550533295
第18轮epoch后，在测试集上,正确个数：1137,总个数：1500,准确率:0.7580000162124634
第18轮epoch后，在验证集上,正确个数：1141,总个数：1500,准确率:0.7606666684150696


100%|██████████| 282/282 [00:25<00:00, 10.96it/s]


第19轮epoch训练时在训练集上,正确个数：17444,总个数：18000,准确率:0.9691111445426941,学习率: 1e-05,loss: 0.09461299329996109
第19轮epoch后，在测试集上,正确个数：1096,总个数：1500,准确率:0.7306666374206543
第19轮epoch后，在验证集上,正确个数：1131,总个数：1500,准确率:0.7540000081062317


100%|██████████| 282/282 [00:25<00:00, 10.85it/s]


第20轮epoch训练时在训练集上,正确个数：17316,总个数：18000,准确率:0.9620000123977661,学习率: 1e-05,loss: 0.11264044791460037
第20轮epoch后，在测试集上,正确个数：1093,总个数：1500,准确率:0.7286666631698608
第20轮epoch后，在验证集上,正确个数：1085,总个数：1500,准确率:0.7233332991600037
